### **E91 Protocol**
Simulation of the E91 Quantum Key Distribution Protocol using Qiskit.

The following code is able to analyze the behaviour of the E91 Protocol under various scenarios.

1. **Ideal Conditions**: No errors in the quantum channel and no eavesdropping. 
2. **Channel Errors**: Errors in the quantum channel but no eavesdropping.
3. **Eavesdropping**: No errors in the quantum channel but Eve gets control of the site-generating entangled photons ($|\psi^-\rangle$) and sends to Alice and Bob a couple of photons in state $|01\rangle$ or $|10\rangle$ with probability $\frac{1}{2}$.

In [1]:
# Importing the required libraries

import math

import random

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt


import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.result import marginal_counts

The following code cell contains the parameters that must be configured to simulate the desired scenario.

In particular, here are reported the different configurations for each of the considered scenarios:

1. **Ideal Conditions**: 
    
    - `n = 1`
    - `theta_values = [np.pi]`
    - `eavesdropping = False`
  
2. **Channel Errors**:

    - `n = 1`
    - `theta_values = [x]` where `x` can be $0 \leq x < \pi$, considering that for `x=0` the resulting state is completely depolarized.
    - `eavesdropping = False`

3. **Eavesdropping**:

    - `n = 1`
    - `theta_values = [np.pi]`
    - `eavesdropping = True`

The `epr_pairs` parameter can be adjusted to experiment with different numbers of EPR pairs generated during the execution of the protocol.

To generate the plots at the bottom of this notebook, as required in the project specifications, a value of `n` greater than 1 is required. Specifically, at least 30 samples are needed for the Central Limit Theorem to apply, since a 95% confidence interval is being calculated. Additionally, to explore different values of $\theta$, and consequently of the Werner parameter $w$, the function `np.linspace(0, np.pi, X)` can be used. This function generates `X` evenly spaced samples between 0 and $\pi$. The parameter `X` determines the number of points on the graph, where the x-axis corresponds to the values of $\theta$ (or equivalently, the Werner parameter derived from these $\theta$ values).

In [2]:
# Number of times the protocol is executed for each setting
n = 30

# Theta values to be used for the generation of the Werner States
theta_values = np.linspace(0, np.pi, 20)

# Whether Eve is eavesdropping or not
eavesdropping = False

# Number of EPR pairs to be generated in one execution of the protocol
epr_pairs = 300